In [ ]:
# 그래프, 수학 기능 추가
# Add graph and math features
import pylab as py
import numpy as np
import numpy.linalg as nl
# 기호 연산 기능 추가
# Add symbolic operation capability
import sympy as sy

In [ ]:
sy.init_printing()

# 룽게-쿠타법 (RK4) : 고차 상미방<br>Runge-Kutta Method (RK4) : Higher Order ODE

## 단진자<br>Simple Pendulum

다음 미분 방정식은 단진자의 운동을 묘사한다.<br>
Following differential equation describes the motion of a simple pendulum.<br>
Ref : Wikipedia contributors, 'Pendulum (mathematics)', Wikipedia, The Free Encyclopedia, 2 June 2018, 13:28 UTC, <https://en.wikipedia.org/w/index.php?title=Pendulum_(mathematics)&oldid=844080803> [accessed 5 August 2018]

$$
\frac{d^2\theta}{dt^2} + \frac{g}{l}sin\theta = 0
$$

상태변수는 다음과 같다고 가정하자.<br>
Let's assume that the state variables are as follows.

$$
\mathbf{x}
=
\begin{pmatrix}
x_0\\
x_1
\end{pmatrix}
=
\begin{pmatrix}
\theta\\
\frac{d}{dt}\theta
\end{pmatrix}
$$

상태변수의 미분은 다음과 같다.<br>Differentiation of the state variables are as follows.

$$
\frac{d}{dt}
\begin{pmatrix}
    x_0\\
    x_1
\end{pmatrix} 
=
\begin{pmatrix}
    x_1\\
    -\frac{g}{l}sinx_0
\end{pmatrix} 
$$

python 함수로 구현해 보면 다음과 같을 것이다.<br>
One possible python implementation would be as follows.

In [ ]:
g_mpsps = 9.8
l_m = 0.3

legends = ('$\\theta(deg)$', '$\\frac{d}{dt}\\theta(deg/s)$')
ylabel = ''

# Initial state
x_0 = np.array([np.deg2rad(90), 0])

def pendulum_NL(x, t):
    """
    Parameters
    ==========
    x: array of theta and d(theta)/dt
    t: time value
    
    Return Value
    ============
    One dimensional array of dx/dt
    """
    
    return np.array([x[1], (-g_mpsps/l_m)*np.sin(x[0])])


룽게-쿠타법을 오일러법, 훈법과 비교해보자.<br>Let's compare the Runge-Kutta method with Euler method, and Heun's method.

In [ ]:
def rk4_step(f, x0, t0, t1):
    """
    One time step of Runge-Kutta method

    f: dx_dt function
    x0 : initial condition
    t0 : this step time
    t1 : next step time
    """
    delta_t = (t1 - t0)
    delta_t_half = delta_t * 0.5
    t_half = t0 + delta_t_half
    
    # Step 1
    s1 = f(x0, t0)

    # Step 2
    s2 = f(x0 + s1 * delta_t_half, t_half)

    # Step 3
    s3 = f(x0 + s2 * delta_t_half, t_half)

    # Step 4
    s4 = f(x0 + s3 * delta_t, t1)

    # Step 5
    s = (1.0 / 6.0) * (s1 + (s2 + s3) * 2 + s4)

    # Step 6
    x1 = x0 + s * delta_t

    return x1


In [ ]:
def rk4(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        x_i_plus_1 = rk4_step(f, x_i, t_i, t_array[k+1])

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list


In [ ]:
def euler(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope
        s_i = f(x_i, t_i)

        # x[i + 1]
        x_i_plus_1 = x_i + s_i * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1)
        
        x_i = x_i_plus_1

    return time_list, result_list


In [ ]:
def heun(f, t_array, x_0):
    time_list = [t_array[0]]
    result_list = [x_0]

    x_i = x_0

    for k, t_i in enumerate(t_array[:-1]):
        # time step
        delta_t = t_array[k+1] - t_array[k]

        # slope at i
        s_i = f(x_i, t_i)

        # x[i + 1] by Forward Euler
        x_i_plus_1 = x_i + s_i * delta_t
        
        # slope at i + 1
        s_i_plus_1 = f(x_i_plus_1, t_array[k+1])
        
        # average of slope
        s_average = (s_i + s_i_plus_1) * 0.5
        
        # x[i + 1] by Modified Euler
        x_i_plus_1_m = x_i + s_average * delta_t

        time_list.append(t_array[k+1])
        result_list.append(x_i_plus_1_m)
        
        x_i = x_i_plus_1_m

    return time_list, result_list


In [ ]:
# Time array
delta_t = 0.001

t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# *** Euler ***
t_euler, x_euler = euler(pendulum_NL, t_sec_array, x_0)
py.plot(t_euler, x_euler, '-', label='Euler')

# *** Heun ***
t_heun, x_heun = heun(pendulum_NL, t_sec_array, x_0)
py.plot(t_heun, x_heun, '-', label='Heun')

# *** RK4 ***
t_rk4, x_rk4 = rk4(pendulum_NL, t_sec_array, x_0)
py.plot(t_rk4, x_rk4, '-', label='RK4')

py.xlabel('t(sec)')
py.ylabel('x(m)')

py.legend(loc=0)
py.grid(True)


## 4차 선형 상미방<br>Fourth Order Linear ODE

$$
         \frac{d^4x}{dt^4} 
         + 12 \frac{d^3x}{dt^3} 
         + 54 \frac{d^2x}{dt^2} 
         + 108 \frac{dx}{dt} 
         + 80 x = 0
$$

$$
\mathbf{q} = \begin{pmatrix}q_0 & q_1 & q_2 & q_3 \end{pmatrix}^T = \begin{pmatrix}x & \frac{dx}{dt} & \frac{d^2x}{dt^2} & \frac{d^3x}{dt^3} \end{pmatrix}^T
$$

$$
\frac{d\mathbf{q}}{dt}
=
\begin{bmatrix}
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
- 80 & - 108 & - 54 & -12
\end{bmatrix}
\begin{pmatrix}
q_0 \\ q_1 \\ q_2 \\ q_3
\end{pmatrix}
=
\mathbf{Aq}
$$


In [ ]:
matrix_A = np.matrix([
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [-80, -108, -54, -12],]
)

legends = (f'$q_{k}$' for k in range(matrix_A.shape[0]))

ylabel = '$\mathbf{q}$'

def fourth_order(q, t):
    """
    Parameters
    ==========
    q: array of q_0, q_1, q_2, and q_3
    t: time value
    
    Return Value
    ============
    One dimensional array of dq/dt
    """

    q_column = np.matrix(q).T
    qdot_column = matrix_A * q_column
    
    qdot_array = np.array(qdot_column.T).flatten()
    
    return qdot_array

# Initial state
x_0 = np.array([1, 0, 0, 0])


In [ ]:
# Time array
delta_t = 0.08
t_sec_array = np.arange(0, 6 + delta_t*0.5, delta_t)

# *** Euler ***
t_euler, x_euler_ode4 = euler(fourth_order, t_sec_array, x_0)

# *** Heun ***
t_heun, x_heun_ode4 = heun(fourth_order, t_sec_array, x_0)

# *** RK4 ***
t_rk4, x_rk4_ode4 = rk4(fourth_order, t_sec_array, x_0)

# Convert to NumPy arrays
x_euler_ode4_array, x_heun_ode4_array, x_rk4_ode4_array = (
    np.array(x_euler_ode4), 
    np.array(x_heun_ode4), 
    np.array(x_rk4_ode4))

py.figure(figsize=(10, 10))

for i_state in range(x_euler_ode4_array.shape[1]):
    py.subplot(x_euler_ode4_array.shape[1], 1, i_state+1)
    py.plot(t_euler, x_euler_ode4_array[:, i_state], '-', label='Euler')
    py.plot(t_heun, x_heun_ode4_array[:, i_state], '-', label='Heun')
    py.plot(t_rk4, x_rk4_ode4_array[:, i_state], '-', label='RK4')

    py.xlabel('t(sec)')
    py.ylabel(f'$q_{i_state}$')

    py.legend(loc=1)
    py.grid(True)


## 부록: 각 상태 변수 점에서 단진자 상태 변수 변화 방향<br>Appendix: Directions of simple pendulum state variable change at each state variable point

각 상태에서 상태변수의 변화의 방향 $\left(\frac{d}{dt}\theta, \frac{d^2}{dt^2}\theta \right)$ 을 표시해 보자.<br>At each state, let's present the direction of state variable change $\left(\frac{d}{dt}\theta, \frac{d^2}{dt^2}\theta \right)$.

In [ ]:
def ode_slopes_2states_cartesian(func, theta_rad_list, theta_dot_rad_list, time_list):
    """
    Plot field of arrows indicating derivatives of the state
    :param func:
    :param theta_rad_list:
    :param theta_dot_rad_list:
    :param time_list:
    :return:
    """

    # cartesian coordinate
    y_rad = np.meshgrid(theta_rad_list, theta_dot_rad_list)

    # derivatives of state at each point
    y_rad_dot = func(y_rad, time_list)

    # color
    color_mesh = np.sqrt(y_rad_dot[0] * y_rad_dot[0] + y_rad_dot[1] * y_rad_dot[1])

    py.figure(figsize=(18, 18))
    py.axis('equal')
    py.quiver(py.rad2deg(y_rad[0]), py.rad2deg(y_rad[1]), py.rad2deg(y_rad_dot[0]), py.rad2deg(y_rad_dot[1]), color_mesh, angles='xy')
    l, r, b, t = py.axis()
    x_span, y2_mesh = r - l, t - b
    py.axis([l - 0.05 * x_span, r + 0.05 * x_span, b - 0.05 * y2_mesh, t + 0.05 * y2_mesh])
    py.grid()



In [ ]:
time_list = []

# list of theta
theta_rad_list = np.deg2rad(np.arange(-540, 540+1, 30))

# list of theta_dot
theta_dot_rad_list = np.deg2rad(np.arange(-540, 540+1, 45))
ode_slopes_2states_cartesian(pendulum_NL, theta_rad_list, theta_dot_rad_list, time_list)

# Convert pendulum results to NumPy arrays
x_euler_array, x_heun_array, x_rk4_array = (
    np.array(x_euler), np.array(x_heun), np.array(x_rk4))

py.plot(py.rad2deg(x_euler_array[:, 0]), py.rad2deg(x_euler_array[:, 1]), '-', label='Euler', alpha=0.7)
py.plot(py.rad2deg(x_heun_array[:, 0]), py.rad2deg(x_heun_array[:, 1]), '.-', label='Heun', alpha=0.7)
py.plot(py.rad2deg(x_rk4_array[:, 0]), py.rad2deg(x_rk4_array[:, 1]), '-', label='RK4')

ax = py.gca()

xlims = py.xlim()
# http://matplotlib.1069221.n5.nabble.com/How-do-I-set-grid-spacing-td9968.html
ax.set_xticks(np.hstack([np.arange(0, xlims[1]+1, 90), np.arange(-90, xlims[0]-1, -90)]))

ylims = py.ylim()
# http://matplotlib.1069221.n5.nabble.com/How-do-I-set-grid-spacing-td9968.html
ax.set_yticks(np.hstack([np.arange(0, ylims[1]+1, 90), np.arange(-90, ylims[0]-1, -90)]))

py.legend(loc=0)
py.xlabel('$\\theta(deg)$')
py.ylabel('$\\frac{d}{dt}\\theta(deg/sec)$')
py.title('Simple pendulum')



## 적용사례 : 오일러 베르누이 외팔보<br>Application Example : Euler-Bernoulli Cantilever

Ref : Wikipedia contributors, 'Euler–Bernoulli beam theory', Wikipedia, The Free Encyclopedia, 19 November 2018, 22:12 UTC, https://en.wikipedia.org/w/index.php?title=Euler%E2%80%93Bernoulli_beam_theory&oldid=869647128 [accessed 24 November 2018]

외팔보는 한쪽 끝에서 처짐과 기울기가 0인 보 이다.<br>A cantilever is a beam with both zero deflection and rotation at one end.

오일러 베르누이 보 이론은 분포하중과 보의 처짐은 다음과 같은 관계를 가진다고 가정한다.<br>
Euler-Bernoulli beam theory assumes that the deflection of a beam and the distributed load has following relationship.

$$
\frac{d^2}{dx^2} \left( EI \frac{d^2w(x)}{dx^2} \right) = q(x)
$$

여기서 $w(x)$와 $q(x)$는 각각 보의 위치 $x$에서의 $z$방향 처짐과 분포하중이다.<br>
Here, $w(x)$ and $q$ are, respectively, $z$ directional deflection and distributed load at a location $x$ of the beam.

단순화 하기 위해 $EI$는 일정하다고 가정하자.<br>
To simplify, let's assume $EI$ is constant.

$$
EI \frac{d^4w(x)}{dx^4} = q(x)
$$

상태 변수 $\mathbb{r}(x)$ 를 다음과 같이 정해 보자.<br>
Let state variable $\mathbb{r}(x)$ as follows.

$$
\mathbb{r}(x) 
= 
\begin{pmatrix}
r_0\\
r_1\\
r_2\\
r_3\\
\end{pmatrix} 
= 
\begin{pmatrix}
w(x) \\
\frac{d}{dx}w(x) \\
\frac{d^2}{dx^2}w(x) \\
\frac{d^3}{dx^3}w(x) \\
\end{pmatrix}
= 
\begin{pmatrix}
w(x) \\
\theta(x) \\
(EI)^{-1}M(x) \\
(EI)^{-1}V(x) \\
\end{pmatrix}
$$

여기서 $\theta(x), M(x), V(x)$ 는 각각 $x$ 에서의 기울기, 모멘트, 전단력이다.<br>
Here, $\theta(x), M(x), V(x)$ are, repectively, slope, moment, and shear force at $x$.

미분해보자.<br>
Let's differentiate.

$$
\frac{d}{dx}\mathbb{r}(x) = \begin{pmatrix}
\frac{d}{dx}w(x) \\
\frac{d^2}{dx^2}w(x) \\
\frac{d^3}{dx^3}w(x) \\
\frac{d^4}{dx^4}w(x) \\
\end{pmatrix}
=
\begin{pmatrix}
r_1\\
r_2\\
r_3\\
(EI)^{-1}q(x) \\
\end{pmatrix}
$$

행렬로 다시 써 보면 다음과 같다.<br>
Let's rewrite using matrices.

$$
\frac{d}{dx}\mathbb{r}(x) = 
\begin{bmatrix}
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0\\
0 & 0 & 0 & 1\\
0 & 0 & 0 & 0\\
\end{bmatrix}
\begin{pmatrix}
r_0\\
r_1\\
r_2\\
r_3 \\
\end{pmatrix}
+
\begin{pmatrix}
0\\
0\\
0\\
(EI)^{-1} \\
\end{pmatrix}
q(x)
$$

$x=0$ 지점에서의 초기조건을 생각해 보자.<br>Let's think about the initial conditions at $x=0$.

$$
\mathbb{r}(0) 
= 
\begin{pmatrix}
w(0) \\
\frac{d}{dx}w(0) \\
\frac{d^2}{dx^2}w(0) \\
\frac{d^3}{dx^3}w(0) \\
\end{pmatrix}
= 
\begin{pmatrix}
w(0) \\
\theta(0) \\
(EI)^{-1}M(0) \\
(EI)^{-1}V(0) \\
\end{pmatrix}
= 
\begin{pmatrix}
0 \\
0 \\
(EI)^{-1}M(0) \\
(EI)^{-1}V(0) \\
\end{pmatrix}
$$

외팔보의 경우, $V(0)$ 는 $q(x)$ 를 $x=0$에서 $x=L$ 까지 적분한 것이다.<br>
In case of a cantilever, $V(0)$ is the integration of $q(x)$ from $x=0$ to $x=L$.

$$
V(0) = \int_{x=0}^{x=L}q(x)dx
$$

$M(0)$는 $q(x)$의 도심 $\bar{x}$에 $V(0)$가 작용할 경우 $x=0$에서의 모멘트이다.<br>
$M(0)$ is the moment at $x=0$ when $V(0)$ is located at the centroid $\bar{x}$ of $q(x)$.

$$
M(0) = \bar{x}V(0)
$$

도심 $\bar{x}$ 는 다음과 같이 구할 수 있다.<br>We can find the cetroid $\bar{x}$ as follows.

$$
\bar{x} = \frac{1}{L}\int_{x=0}^{x=L}xq(x)dx
$$

또는 모멘트 $M(0)$는 다음과 같이 구할 수 있다.<br>
Or we can find the moment $M(0)$ as follows.

$$
M(0) = \int_{x=0}^{x=L}xq(x)dx
$$

## 도전 과제<br>Try This

다음 2계 선형 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Find the numerical solutions of the following second order linear ordinary equation using RK4 Method:

$$
\begin{align}
\frac{d^2}{dt^2}x(t) + \frac{d}{dt}x(t) + 4x(t) &= 0 \\
x(0) &= 0 \\
\frac{d}{dt}x(0) &= 1
\end{align}
$$

다음 2계 선형 상미분 방정식의 수치해를 RK4법으로 구하시오:<br>
Find the numerical solutions of the following second order linear ordinary equation using RK4 Method:

$$
\begin{align}
\frac{d^2}{dt^2}x(t) + 2\frac{d}{dt}x(t) + x(t) &= 0 \\
x(0) &= 0 \\
\frac{d}{dt}x(0) &= 1
\end{align}
$$